In [32]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 100_0000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2021-09-14  363.0  356.0  361.0  358.0   66934700.0      358.0
2021-09-15  367.0  361.0  364.0  362.0   54868371.0      362.0
2021-09-16  371.0  360.0  360.0  367.0   52270565.0      367.0
2021-09-17  367.0  362.0  366.0  362.0   43735573.0      362.0
2021-09-21  379.0  373.0  378.0  378.0   83213264.0      378.0
...           ...    ...    ...    ...          ...        ...
2022-09-08  367.0  357.0  367.0  357.0  113510594.0      357.0
2022-09-09  357.0  352.0  354.0  353.0   99795902.0      353.0
2022-09-12  348.0  343.0  346.0  346.0   73706891.0      346.0
2022-09-13  346.0  342.0  345.0  344.0   46433034.0      344.0
2022-09-14  364.0  357.0  363.0  360.0  101181564.0      360.0

[246 rows x 6 columns]


**ボリンジャーバンドとRSI組み合わせ**

In [ ]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.276423
Equity Final [$]                  1000496.261
Equity Peak [$]                  1225630.8635
Return [%]                           0.049626
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                    0.050837
Volatility (Ann.) [%]               24.867373
Sharpe Ratio                         0.002044
Sortino Ratio                        0.003124
Calmar Ratio                         0.002036
Max. Drawdown [%]                  -24.969797
Avg. Drawdown [%]                   -6.865166
Max. Drawdown Duration       86 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                    8
Win Rate [%]                             50.0
Best Trade [%]                       9.558695
Worst Trade [%]                     -9.829721
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,127,159,397.96020,436.0,95555.97760,0.095587,2022-03-24,2022-05-13,50 days
1,2838,175,176,385.90080,372.0,-39450.47040,-0.036022,2022-06-06,2022-06-07,1 days
2,2825,176,178,373.84140,366.0,-22151.95500,-0.020975,2022-06-07,2022-06-09,2 days
3,2811,178,207,367.81170,375.0,20206.31130,0.019543,2022-06-09,2022-07-21,42 days
4,2797,207,208,376.85625,370.0,-19176.93125,-0.018193,2022-07-21,2022-07-22,1 days
5,2783,208,209,371.83150,373.0,3251.93550,0.003143,2022-07-22,2022-07-25,3 days
6,2769,209,226,374.84635,338.0,-102027.54315,-0.098297,2022-07-25,2022-08-18,24 days
7,2756,226,245,339.67310,363.0,64288.93640,0.068675,2022-08-18,2022-09-14,27 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.682927
Equity Final [$]                 1310968.5235
Equity Peak [$]                  1310968.5235
Return [%]                          31.096852
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   31.965488
Volatility (Ann.) [%]               32.368294
Sharpe Ratio                         0.987556
Sortino Ratio                         2.24024
Calmar Ratio                         2.346235
Max. Drawdown [%]                  -13.624164
Avg. Drawdown [%]                   -4.699335
Max. Drawdown Duration       56 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                      13.113013
Worst Trade [%]                     -8.573212
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2646,33,42,377.86120,370.0,-20800.73520,-0.020804,2021-11-02,2021-11-16,14 days
1,2633,42,50,371.83150,403.0,82066.66050,0.083824,2021-11-16,2021-11-29,13 days
2,2831,75,85,374.84635,424.0,139153.98315,0.131130,2022-01-05,2022-01-20,15 days
3,2771,124,127,433.13345,396.0,-102896.78995,-0.085732,2022-03-18,2022-03-24,6 days
4,2757,127,141,397.96020,425.0,74548.72860,0.067946,2022-03-24,2022-04-13,20 days
5,2894,148,159,404.99485,436.0,89728.90410,0.076557,2022-04-22,2022-05-13,21 days
6,3211,171,175,392.93545,384.0,-28691.72995,-0.022740,2022-05-31,2022-06-06,6 days
7,3195,175,176,385.90080,372.0,-44413.05600,-0.036022,2022-06-06,2022-06-07,1 days
8,3179,176,178,373.84140,366.0,-24927.81060,-0.020975,2022-06-07,2022-06-09,2 days
9,3164,178,182,367.81170,410.0,133483.78120,0.114701,2022-06-09,2022-06-15,6 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2537,51,127,394.0398,396.0,-4973.0274,-0.004975,2021-11-30,2022-03-24,114 days
1,2500,127,159,397.9602,436.0,95099.5000,0.095587,2022-03-24,2022-05-13,50 days
2,-2512,159,175,433.8418,384.0,125202.6016,0.114885,2022-05-13,2022-06-06,24 days
3,3149,175,245,385.9008,363.0,-72114.6192,-0.059344,2022-06-06,2022-09-14,100 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   86.585366
Equity Final [$]                1900419.70255
Equity Peak [$]                 1900419.70255
Return [%]                           90.04197
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   93.041522
Volatility (Ann.) [%]                80.18508
Sharpe Ratio                         1.160335
Sortino Ratio                        3.788357
Calmar Ratio                         4.171789
Max. Drawdown [%]                  -22.302549
Avg. Drawdown [%]                   -4.879621
Max. Drawdown Duration       70 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                   13
Win Rate [%]                        76.923077
Best Trade [%]                      13.113013
Worst Trade [%]                     -2.156619
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2646,33,50,377.86120,403.0,66517.26480,0.066529,2021-11-02,2021-11-29,27 days
1,-2659,50,75,401.00515,373.0,74465.69385,0.069837,2021-11-29,2022-01-05,37 days
2,3043,75,85,374.84635,424.0,149574.55695,0.131130,2022-01-05,2022-01-20,15 days
3,-3058,85,124,421.90120,431.0,-27824.13040,-0.021566,2022-01-20,2022-03-18,57 days
4,2915,124,141,433.13345,425.0,-23709.00675,-0.018778,2022-03-18,2022-04-13,26 days
5,-2929,141,148,422.89625,403.0,58276.11625,0.047048,2022-04-13,2022-04-22,9 days
6,3203,148,159,404.99485,436.0,99309.49545,0.076557,2022-04-22,2022-05-13,21 days
7,-3219,159,171,433.84180,391.0,137907.75420,0.098750,2022-05-13,2022-05-31,18 days
8,3905,171,182,392.93545,410.0,66637.06775,0.043428,2022-05-31,2022-06-15,15 days
9,-3924,182,207,407.97050,375.0,129376.24200,0.080816,2022-06-15,2022-07-21,36 days


<Strategy BBandRSI_WithShortPosition(upper_bound=60,lower_bound=40,rsi_window=10)>

**MACDとRSIの組み合わせ**

In [29]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    upper_bound = range(50, 85, 5),\
    lower_bound = range(10, 45, 5),\
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2017-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                    1000000.0
Equity Peak [$]                     1000000.0
Return [%]                                0.0
Buy & Hold Return [%]               -7.019231
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 525 configurations.
  output = _optimize_grid()


Start                     2017-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                    1.696284
Equity Final [$]                 1221055.7855
Equity Peak [$]                  1251075.7855
Return [%]                          22.105579
Buy & Hold Return [%]               -7.019231
Return (Ann.) [%]                    4.149064
Volatility (Ann.) [%]                8.204732
Sharpe Ratio                         0.505692
Sortino Ratio                        1.149515
Calmar Ratio                          0.57438
Max. Drawdown [%]                   -7.223552
Avg. Drawdown [%]                   -3.331627
Max. Drawdown Duration      867 days 00:00:00
Avg. Drawdown Duration      149 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      22.292968
Worst Trade [%]                     22.292968
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,79,638,658,12551.8255,15350.0,221055.7855,0.22293,2020-04-02,2020-05-01,29 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-49,35,1237,20279.119,14420.0,287096.831,0.288924,2017-11-02,2022-09-14,1777 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 525 configurations.
  output = _optimize_grid()


Start                     2017-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   94.911147
Equity Final [$]                   1804219.05
Equity Peak [$]                    2576779.05
Return [%]                          80.421905
Buy & Hold Return [%]               -7.019231
Return (Ann.) [%]                   12.763549
Volatility (Ann.) [%]               58.321891
Sharpe Ratio                         0.218847
Sortino Ratio                        0.493467
Calmar Ratio                         0.287666
Max. Drawdown [%]                  -44.369346
Avg. Drawdown [%]                  -11.617582
Max. Drawdown Duration      860 days 00:00:00
Avg. Drawdown Duration      123 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      40.454777
Worst Trade [%]                      5.591309
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-47,63,638,20975.6540,12490.0,398825.7380,0.404548,2017-12-12,2020-04-02,842 days
1,111,638,658,12551.8255,15350.0,310597.3695,0.222930,2020-04-02,2020-05-01,29 days
2,-111,658,1237,15274.0175,14420.0,94795.9425,0.055913,2020-05-01,2022-09-14,866 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=20,MACDlong=50,MACDsignal=10,upper_bound=60,lower_bound=40,rsi_window=10)>

**MACDとボリンジャーバンドの組み合わせ**

In [35]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   17.073171
Equity Final [$]                 917270.95115
Equity Peak [$]                 1089749.97375
Return [%]                          -8.272905
Buy & Hold Return [%]                0.558659
Return (Ann.) [%]                   -8.465893
Volatility (Ann.) [%]               19.565045
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.153708
Avg. Drawdown [%]                  -10.666584
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                       9.414152
Worst Trade [%]                    -12.705748
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,51,59,397.96020,397.0,-2412.02240,-0.002413,2021-11-30,2021-12-10,10 days
1,2341,85,89,426.09880,430.0,9132.70920,0.009156,2022-01-20,2022-01-26,6 days
2,2329,89,97,432.12850,422.0,-23589.27650,-0.023439,2022-01-26,2022-02-07,12 days
3,2164,109,116,454.23740,497.0,92538.26640,0.094142,2022-02-25,2022-03-08,11 days
4,2153,116,123,499.46015,436.0,-136629.70295,-0.127057,2022-03-08,2022-03-17,9 days
5,2143,159,163,438.15820,428.0,-21769.02260,-0.023184,2022-05-13,2022-05-19,6 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   20.325203
Equity Final [$]                  860246.0807
Equity Peak [$]                  1103450.4197
Return [%]                         -13.975392
Buy & Hold Return [%]                0.558659
Return (Ann.) [%]                  -14.290663
Volatility (Ann.) [%]               18.818742
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -22.04035
Avg. Drawdown [%]                  -10.851525
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                       9.414152
Worst Trade [%]                    -13.706829
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,51,62,397.96020,402.0,10147.9776,0.010151,2021-11-30,2021-12-15,15 days
1,2370,85,89,426.09880,430.0,9245.8440,0.009156,2022-01-20,2022-01-26,6 days
2,2359,89,98,432.12850,422.0,-23893.1315,-0.023439,2022-01-26,2022-02-08,13 days
3,2191,109,116,454.23740,497.0,93692.8566,0.094142,2022-02-25,2022-03-08,11 days
4,2180,116,124,499.46015,431.0,-149243.1270,-0.137068,2022-03-08,2022-03-18,10 days
5,2145,159,166,438.15820,401.0,-79704.3390,-0.084805,2022-05-13,2022-05-24,11 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=30,MACDsignal=10)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   79.268293
Equity Final [$]                 995613.31775
Equity Peak [$]                  1058451.9726
Return [%]                          -0.438668
Buy & Hold Return [%]                0.558659
Return (Ann.) [%]                   -0.449343
Volatility (Ann.) [%]               35.036658
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.366348
Avg. Drawdown [%]                   -9.606002
Max. Drawdown Duration      252 days 00:00:00
Avg. Drawdown Duration       58 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      16.328948
Worst Trade [%]                    -10.502741
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2537,51,85,394.03980,424.0,-76009.02740,-0.076033,2021-11-30,2022-01-20,51 days
1,-2190,85,89,421.90120,430.0,-17736.37200,-0.019196,2022-01-20,2022-01-26,6 days
2,-2118,89,109,427.87150,452.0,-51104.16300,-0.056392,2022-01-26,2022-02-25,30 days
3,-1901,109,116,449.76260,497.0,-89798.29740,-0.105027,2022-02-25,2022-03-08,11 days
4,-1547,116,159,494.53985,436.0,90561.14795,0.118372,2022-03-08,2022-05-13,66 days
5,-1972,159,245,433.84180,363.0,139700.02960,0.163289,2022-05-13,2022-09-14,124 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   65.447154
Equity Final [$]                 1077578.9421
Equity Peak [$]                  1143763.9421
Return [%]                           7.757894
Buy & Hold Return [%]                0.558659
Return (Ann.) [%]                    7.954447
Volatility (Ann.) [%]               32.783392
Sharpe Ratio                         0.242636
Sortino Ratio                        0.389567
Calmar Ratio                         0.376583
Max. Drawdown [%]                  -21.122692
Avg. Drawdown [%]                   -7.870405
Max. Drawdown Duration      134 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                      16.328948
Worst Trade [%]                    -10.502741
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2370,85,89,421.90120,430.0,-19194.1560,-0.019196,2022-01-20,2022-01-26,6 days
1,-2292,89,109,427.87150,452.0,-55302.5220,-0.056392,2022-01-26,2022-02-25,30 days
2,-2057,109,116,449.76260,497.0,-97167.3318,-0.105027,2022-02-25,2022-03-08,11 days
3,-1674,116,159,494.53985,436.0,97995.7089,0.118372,2022-03-08,2022-05-13,66 days
4,-2135,159,245,433.84180,363.0,151247.2430,0.163289,2022-05-13,2022-09-14,124 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=60,MACDsignal=10)>